<a href="https://colab.research.google.com/github/affan002/DimABSA-SemEval-task03/blob/main/Task1_semevel_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing libraries
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
#import data
import pandas as pd
from datasets import Dataset
url = "https://raw.githubusercontent.com/affan002/DimABSA-SemEval-task03/refs/heads/main/train/eng_laptop_train_alltasks.jsonl?token=GHSAT0AAAAAADIPLQETWIS2B4GDCX5HATLE2GTAOKA"
df = pd.read_json(url, lines=True)

rows = []
for _, row in df.iterrows():
    text = row["Text"]
    for quad in row["Quadruplet"]:
        aspect = quad["Aspect"]
        va = quad["VA"]
        valence, arousal = map(float, va.split("#"))
        rows.append({
            "Text": text,
            "Aspect": aspect,
            "VA": va,
            "Valence": valence,
            "Arousal": arousal
        })

raw_datasets = Dataset.from_pandas(pd.DataFrame(rows))


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(batch):
    tokenized = tokenizer(
        list(batch["Aspect"]),
        list(batch["Text"]),
        truncation=True
    )
    valence = [float(v.split("#")[0]) for v in batch["VA"]]
    arousal = [float(v.split("#")[1]) for v in batch["VA"]]
    tokenized["labels"] = list(zip(valence, arousal))
    return tokenized

# Batched map works now
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print(tokenized_datasets[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/5773 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'Text': 'this unit is ` ` pretty ` ` and stylish , so my high school daughter was attracted to it for that reason .', 'Aspect': 'unit', 'VA': '7.12#7.12', 'Valence': 7.12, 'Arousal': 7.12, 'input_ids': [1, 1692, 2, 291, 1692, 269, 9043, 9043, 890, 9043, 9043, 263, 5504, 366, 324, 312, 459, 563, 1816, 284, 7427, 264, 278, 270, 272, 919, 323, 2], 'token_type_ids': [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [7.12, 7.12]}


In [ ]:
# Drop raw columns we don't need for training
tokenized_datasets = tokenized_datasets.remove_columns(
    ["Text", "Aspect", "VA", "Valence", "Arousal"]
)

# Convert dataset to PyTorch tensors
tokenized_datasets.set_format("torch")

# Check final columns
print(tokenized_datasets.column_names)

['input_ids', 'token_type_ids', 'attention_mask', 'labels']


In [ ]:
# Split into train/validation/test
# First: train + temp (where temp will be split further into val + test)
dataset_splits = tokenized_datasets.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset_splits["train"]
temp_dataset = dataset_splits["test"]

# Now split temp into validation and test (50/50 → 10% val, 10% test overall)
temp_splits = temp_dataset.train_test_split(test_size=0.5, seed=42)

eval_dataset = temp_splits["train"]   # validation set
test_dataset = temp_splits["test"]    # final test set

# Make DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset, shuffle=True, batch_size=16, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=16, collate_fn=data_collator
)
test_dataloader = DataLoader(
    test_dataset, batch_size=16, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([16, 35]),
 'token_type_ids': torch.Size([16, 35]),
 'attention_mask': torch.Size([16, 35]),
 'labels': torch.Size([16, 2])}

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Tell HF to treat this as regression with 2 outputs (Valence, Arousal)
config = AutoConfig.from_pretrained(checkpoint, num_labels=2, problem_type="regression")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Pass the batch through the model to get outputs (predictions + loss)
outputs = model(**batch)

# Print:
#  - outputs.loss → the current Mean Squared Error (MSE) loss
#  - outputs.logits.shape → the shape of the predictions tensor (batch_size × 2)
print(outputs.loss, outputs.logits.shape)

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

tensor(46.0921, grad_fn=<MseLossBackward0>) torch.Size([16, 2])


In [ ]:
from torch.optim import AdamW

# Define optimizer that updates the model's parameters
optimizer = AdamW(model.parameters(), lr=3e-5,weight_decay=0.01)
# ⚠️ The only thing you might tune later:
# Learning rate (lr) → try 5e-5, 3e-5, or 1e-5 to see which gives better results.
# Weight decay → if overfitting, you can add e.g. weight_decay=0.01.

In [ ]:
from transformers import get_scheduler

# Train for more epochs since dataset is small
num_epochs = 8   # you can try 5, 8, or even 10

# Total number of training steps
num_training_steps = num_epochs * len(train_dataloader)

# Warmup = 10% of training steps
num_warmup_steps = int(0.1 * num_training_steps)

# Define learning rate scheduler
lr_scheduler = get_scheduler(
    "linear",                # linear decay schedule
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,   # gradual warmup
    num_training_steps=num_training_steps,
)

print(f"Total steps: {num_training_steps}, Warmup steps: {num_warmup_steps}")


Total steps: 2312, Warmup steps: 231


In [ ]:
import torch

# Step 1: Choose device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Step 2: Move the model to the chosen device
model.to(device)
device


device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

# Create a progress bar for visual feedback
progress_bar = tqdm(range(num_training_steps))

model.train()  # Set model to training mode
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}  # Move inputs & labels to GPU/CPU
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()  # Compute

        optimizer.step()       # Update model weights using gradients
        lr_scheduler.step()    # Update learning rate according to schedule
        optimizer.zero_grad()  # Reset gradients for next step
        progress_bar.update(1) # Advance progress bar




  0%|          | 0/2312 [00:00<?, ?it/s]

In [ ]:
import torch
import numpy as np

model.eval()  # evaluation mode

valence_preds_all = []
arousal_preds_all = []
valence_refs_all = []
arousal_refs_all = []

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    predictions = outputs.logits  # [batch_size, 2]
    references = batch["labels"]   # [batch_size, 2]

    # Separate valence and arousal
    valence_preds_all.append(predictions[:, 0].cpu().numpy())
    arousal_preds_all.append(predictions[:, 1].cpu().numpy())
    valence_refs_all.append(references[:, 0].cpu().numpy())
    arousal_refs_all.append(references[:, 1].cpu().numpy())

# Concatenate all batches
valence_preds_all = np.concatenate(valence_preds_all)
arousal_preds_all = np.concatenate(arousal_preds_all)
valence_refs_all = np.concatenate(valence_refs_all)
arousal_refs_all = np.concatenate(arousal_refs_all)

# Compute MSE and MAE for each dimension
valence_mse = ((valence_preds_all - valence_refs_all) ** 2).mean()
arousal_mse = ((arousal_preds_all - arousal_refs_all) ** 2).mean()

valence_mae = np.abs(valence_preds_all - valence_refs_all).mean()
arousal_mae = np.abs(arousal_preds_all - arousal_refs_all).mean()

# ✅ Add RMSE for each dimension
valence_rmse = np.sqrt(valence_mse)
arousal_rmse = np.sqrt(arousal_mse)

print(f"Valence → MSE: {valence_mse:.4f}, MAE: {valence_mae:.4f}, RMSE: {valence_rmse:.4f}")
print(f"Arousal → MSE: {arousal_mse:.4f}, MAE: {arousal_mae:.4f}, RMSE: {arousal_rmse:.4f}")


Valence → MSE: 0.7706, MAE: 0.6524, RMSE: 0.8778
Arousal → MSE: 0.6303, MAE: 0.6211, RMSE: 0.7939


In [ ]:
# Normalized DimASR RMSE
D_max = 8**2 + 8**2  # 128

# Clip predictions to [1,9] and round to 2 decimals
valence_preds_clipped = np.clip(np.round(valence_preds_all, 2), 1, 9)
arousal_preds_clipped = np.clip(np.round(arousal_preds_all, 2), 1, 9)

# Compute squared distances in VA space
squared_distances = (valence_preds_clipped - valence_refs_all) ** 2 + \
                    (arousal_preds_clipped - arousal_refs_all) ** 2

# Normalized RMSE
dimasr_rmse = np.sqrt((squared_distances / D_max).mean())

print(f"DimASR Normalized RMSE: {dimasr_rmse:.4f}")


DimASR Normalized RMSE: 0.1046


In [ ]:
#code to save model in local
save_path = "./deberta-v3-base-finetuned-task1-semeval"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./deberta-v3-base-finetuned-task1-semeval/tokenizer_config.json',
 './deberta-v3-base-finetuned-task1-semeval/special_tokens_map.json',
 './deberta-v3-base-finetuned-task1-semeval/spm.model',
 './deberta-v3-base-finetuned-task1-semeval/added_tokens.json',
 './deberta-v3-base-finetuned-task1-semeval/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Paths
finetuned_path = "./deberta-v3-base-finetuned-task1-semeval"
fresh_checkpoint = "microsoft/deberta-v3-base"

# Load fine-tuned model + tokenizer
finetuned_model = AutoModelForSequenceClassification.from_pretrained(finetuned_path)
finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_path)

# Load fresh base BERT (not fine-tuned)
fresh_model = AutoModelForSequenceClassification.from_pretrained(
    fresh_checkpoint,
    num_labels=2,            # same as your fine-tuned
    problem_type="regression"
)
fresh_tokenizer = AutoTokenizer.from_pretrained(fresh_checkpoint)

# Example input
text = "for now i ' m okay with upping the experience & device to 3 out of 5 stars "
aspect = "device"

# Tokenize the same way you trained
inputs = finetuned_tokenizer(aspect, text, return_tensors="pt", truncation=True)

# Predictions
with torch.no_grad():
    finetuned_preds = finetuned_model(**inputs).logits.squeeze().tolist()
    fresh_preds = fresh_model(**inputs).logits.squeeze().tolist()

print("Input Text:", text)
print("Aspect:", aspect)
print("Fine-tuned predictions → Valence, Arousal:", finetuned_preds)
print("Fresh pretrained predictions → Valence, Arousal:", fresh_preds)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Input Text: for now i ' m okay with upping the experience & device to 3 out of 5 stars 
Aspect: device
Fine-tuned predictions → Valence, Arousal: [5.5924177169799805, 5.536684036254883]
Fresh pretrained predictions → Valence, Arousal: [0.05295417830348015, -0.03205728530883789]


From here onwards its to upload model on hugging face so change your account for it

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()


In [ ]:
# from huggingface_hub import HfApi

# api = HfApi()
# repo_id = "hassanshahzad2003/deberta-v3-base-finetuned-task1-semeval"

# # Create repo if not exists
# api.create_repo(repo_id=repo_id, repo_type="model", private=False)

# # Upload all files in the folder
# api.upload_folder(
#     folder_path="/content/deberta-v3-base-finetuned-task1-semeval",
#     repo_id=repo_id
# )


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...d-task1-semeval/spm.model:  95%|#########5| 2.35MB / 2.46MB            

  ...semeval/model.safetensors:   0%|          |  591kB /  738MB            

CommitInfo(commit_url='https://huggingface.co/hassanshahzad2003/deberta-v3-base-finetuned-task1-semeval/commit/f8b96456ec29e28d172e098684c8bc45dc5e2d38', commit_message='Upload folder using huggingface_hub', commit_description='', oid='f8b96456ec29e28d172e098684c8bc45dc5e2d38', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hassanshahzad2003/deberta-v3-base-finetuned-task1-semeval', endpoint='https://huggingface.co', repo_type='model', repo_id='hassanshahzad2003/deberta-v3-base-finetuned-task1-semeval'), pr_revision=None, pr_num=None)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Replace with your repo name
# model_name = "hassanshahzad2003/deberta-v3-base-finetuned-task1-semeval"

# # Load tokenizer and model from Hugging Face Hub
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)

# # Example usage
# text = "This laptop has amazing battery life!"
# inputs = tokenizer(text, return_tensors="pt")

# with torch.no_grad():
#     outputs = model(**inputs)

# print(outputs.logits)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tensor([[8.4160, 8.4373]])
